In [11]:
!pip -q install evaluate seqeval

In [12]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset
import numpy as np

In [13]:
model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=7)

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [15]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [16]:
# 模型测试
message= "命名实体识别"
label = torch.tensor([0,1,0,2,5,4])

model_input = tokenizer([message], return_tensors='pt')
result = model(**model_input)

print(result.loss)
print(result.logits)

None
tensor([[[ 0.3328, -0.2662, -0.1428, -0.1978, -0.4868, -1.4934,  0.7539],
         [ 0.4074, -0.7107,  0.4805, -0.3104, -0.3438, -1.0272,  0.6406],
         [-0.0743, -0.1020,  0.4508, -0.0212, -0.3985, -0.9073,  0.0806],
         [-0.3371, -0.3284,  0.4692, -0.1262, -0.3711, -0.8790, -0.3829],
         [-0.1737, -0.2531,  0.5247, -0.2002, -0.1514, -1.2913,  0.0605],
         [-0.6175,  0.0764,  0.6844, -0.2436, -0.0042, -1.0078, -0.1176],
         [-0.0885, -0.2327, -0.2008, -1.1142, -0.4992, -1.4460, -0.0302],
         [-0.3209,  0.0764,  0.8272, -0.0095, -0.0070, -0.9031,  0.3057]]],
       grad_fn=<ViewBackward0>)


In [17]:
# 加载hf中dataset
ds = load_dataset('doushabao4766/msra_ner_k_V3')
ds

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-42717a92413393f9.parquet:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

(…)-00000-of-00001-8899cab5fdab45bc.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge'],
        num_rows: 3443
    })
})

In [18]:
ds["train"]

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'knowledge'],
    num_rows: 45001
})

## 实体映射数据集词典准备

In [19]:
# entity_index
entites = ['O'] + list({'movie', 'name', 'game', 'address', 'position', \
           'company', 'scene', 'book', 'organization', 'government'})
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())

entity_index = {entity:i for i, entity in enumerate(entites)}


In [20]:
entity_index

{'O': 0,
 'organization': 1,
 'company': 2,
 'government': 3,
 'book': 4,
 'position': 5,
 'movie': 6,
 'game': 7,
 'name': 8,
 'scene': 9,
 'address': 10}

In [22]:
tags

['O',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'B-COMPANY',
 'I-COMPANY',
 'B-GOVERNMENT',
 'I-GOVERNMENT',
 'B-BOOK',
 'I-BOOK',
 'B-POSITION',
 'I-POSITION',
 'B-MOVIE',
 'I-MOVIE',
 'B-GAME',
 'I-GAME',
 'B-NAME',
 'I-NAME',
 'B-SCENE',
 'I-SCENE',
 'B-ADDRESS',
 'I-ADDRESS']

In [30]:
# 训练集
for row in ds['train']:
    print(row)
    print(row['tokens'])
    # print(row['tokens'])
    # print(row['tokens'])
    # print(row['ner_tags'])
    # print(row['ent_tag'])
    
    break

{'id': '0', 'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'knowledge': ''}
['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']


In [20]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'ent_tag'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'ent_tag'],
        num_rows: 3443
    })
})

中文bert分词在日期时间和英文转换token过程中，出现合并。影响ner标注准确性。

In [23]:
token_index = tokenizer.encode('2000年2月add', add_special_tokens=False)
print(token_index)
tokens = tokenizer.decode(token_index)
print(tokens)

[8202, 2399, 123, 3299, 10253]
2000 年 2 月 add


In [24]:
input_data = tokenizer(['2000年2月add'], add_special_tokens=False, truncation=True)
print(input_data)

input_data.word_ids(0) # 返回批次中指定token对应原始文本的索引映射

{'input_ids': [[8202, 2399, 123, 3299, 10253]], 'token_type_ids': [[0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


[0, 1, 2, 3, 4]

In [25]:
def data_input_proc(item):
    # 处理批处理数据（batched=True时item是字典，每个值都是列表）
    texts = []
    for tokens in item['tokens']:
        if len(tokens) == 0:
            texts.append('')
            continue
        # 确保每个样本的token列表被正确转换为字符串
        if isinstance(tokens[0], list):
            tokens = [t for sublist in tokens for t in sublist]
        texts.append(''.join(tokens))
    input_data = tokenizer(texts, truncation=True, add_special_tokens=False, max_length=512)
    
    # 获取word_ids映射（批处理）
    # word_ids_batch = [input_data.word_ids(batch_index=i) for i in range(len(texts))]
    
    # 调整标签序列
    adjust_labels = []
    # 上一步骤生成ner_tag中索引和token对齐
    for k in range(len(input_data['input_ids'])):
        # 每条记录token对应word_ids
        word_ids = input_data.word_ids(k)
        # 批次ner_tag长度和token长度对齐
        tags = item['ner_tags'][k]
        
        adjusted_label_ids = []
        i, prev_wid = -1,-1
        for wid in word_ids:
            if (wid != prev_wid):   #  word_ids [1,1,1,2,3,4,5] -> [0,1,2,3,4,5,6]
                i += 1 # token对应检索位置+1
                prev_wid = wid
            adjusted_label_ids.append(tags[i])
        adjust_labels.append(adjusted_label_ids)                
    # 修正后label添加到input_data
    input_data['labels'] = adjust_labels
    return input_data

# 应用处理函数
ds2 = ds.map(data_input_proc, batched=True)

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [26]:
ds2

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3443
    })
})

In [27]:
# 训练集
for row in ds2['train']:
    print(row)
    
    break

{'id': '0', 'tokens': ['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'knowledge': '', 'input_ids': [2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [28]:
# 训练集
for row in ds2['train']:
    print(row['id'])
    print(row['tokens'])
    print(row['ner_tags'])
    print(row['input_ids'])
    # print(row['token_type_ids'])
    print(row['attention_mask'])
    print(row['labels'])
    print()
    print(len(row['tokens']))
    print(len(row['ner_tags']))
    print(len(row['input_ids']))
    # print(row['token_type_ids'])
    print(len(row['attention_mask']))
    print(len(row['labels']))
    break

0
['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

5

In [29]:
# 记录转换为pytorch
ds2.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
# ds_new = ds2.with_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [30]:
for item in ds2['train']:
    print(item)
    break

{'input_ids': tensor([2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636,  674, 1036, 4997,
        2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768,
        7599, 3198, 8024,  791, 1921, 3300, 3119, 5966,  817,  966, 4638,  741,
         872, 3766,  743, 8024, 3209, 3189, 2218, 1373,  872, 2637,  679, 2496,
        1159, 8013]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]), 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])}


## 模型手动训练


In [27]:
# 废弃
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 3,    # 训练 epoch
    save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=32,  # 训练批次
    per_device_eval_batch_size=32,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

In [31]:
[name for name, params in model.named_parameters()]

['bert.embeddings.word_embeddings.weight',
 'bert.embeddings.position_embeddings.weight',
 'bert.embeddings.token_type_embeddings.weight',
 'bert.embeddings.LayerNorm.weight',
 'bert.embeddings.LayerNorm.bias',
 'bert.encoder.layer.0.attention.self.query.weight',
 'bert.encoder.layer.0.attention.self.query.bias',
 'bert.encoder.layer.0.attention.self.key.weight',
 'bert.encoder.layer.0.attention.self.key.bias',
 'bert.encoder.layer.0.attention.self.value.weight',
 'bert.encoder.layer.0.attention.self.value.bias',
 'bert.encoder.layer.0.attention.output.dense.weight',
 'bert.encoder.layer.0.attention.output.dense.bias',
 'bert.encoder.layer.0.attention.output.LayerNorm.weight',
 'bert.encoder.layer.0.attention.output.LayerNorm.bias',
 'bert.encoder.layer.0.intermediate.dense.weight',
 'bert.encoder.layer.0.intermediate.dense.bias',
 'bert.encoder.layer.0.output.dense.weight',
 'bert.encoder.layer.0.output.dense.bias',
 'bert.encoder.layer.0.output.LayerNorm.weight',
 'bert.encoder.layer

### Model

In [32]:
# dataLoader
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim

train_dl = DataLoader(ds2['train'], shuffle=True, batch_size=16)


# 模型创建
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                        num_labels=21,
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model.to('cuda')

# 模型参数分组
param_optimizer = list(model.named_parameters())
bert_params, classifier_params = [],[]

for name,params in param_optimizer:
    if 'bert' in name:
        bert_params.append(params)
    else:
        classifier_params.append(params)

param_groups = [
    {'params':bert_params, 'lr':1e-5},
    {'params':classifier_params, 'weight_decay':0.1, 'lr':1e-3}
]

# optimizer
optimizer = optim.AdamW(param_groups) # 优化器

# 学习率调度器
train_steps = len(train_dl) * 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=100, 
                                            num_training_steps=train_steps)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 支持混合精度训练

In [33]:
# dataLoader
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim

train_dl = DataLoader(ds2['train'], shuffle=True, batch_size=16)


# 模型创建
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                        num_labels=21,
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model.to('cuda')

# 模型参数分组
param_optimizer = list(model.named_parameters())
bert_params, classifier_params = [],[]

for name,params in param_optimizer:
    if 'bert' in name:
        bert_params.append(params)
    else:
        classifier_params.append(params)

param_groups = [
    {'params':bert_params, 'lr':1e-5},
    {'params':classifier_params, 'weight_decay':0.1, 'lr':1e-3}
]

# optimizer
optimizer = optim.AdamW(param_groups) # 优化器

# 学习率调度器
train_steps = len(train_dl) * 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=100, 
                                            num_training_steps=train_steps)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 支持分布式训练

In [34]:
%%writefile ddp_simple.py

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler

# 设置分布式环境
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

# 清理分布式环境
def cleanup():
    dist.destroy_process_group()

# 定义训练循环
def train(rank, world_size):
    setup(rank, world_size)
    
    # 定义模型并将其移动到对应的 GPU 设备端
    model = models.resnet50().to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    
    # 损失函数及优化器
    criterion = nn.CrossEntropyLoss().to(rank)
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.01)
    
    # 定义数据集Dataset的转换和图像增强
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    # 分布式训练采样器
    sampler = DistributedSampler(dataset, num_replicas=world_size, rank=rank)
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=32)
    
    # 在训练开始时创建一次
    scaler = torch.cuda.amp.GradScaler()
    
    for epoch in range(10):
        ddp_model.train()
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(rank), labels.to(rank)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = ddp_model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(ddp_model.parameters(), 1)
            
            scaler.step(optimizer)
            scaler.update()
            
#             loss.backward()
#             optimizer.step()
            print(f"Rank {rank}, Epoch {epoch}, Loss: {loss.item()}")

    cleanup()
    
def main():
    world_size = torch.cuda.device_count()
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)

if __name__ == "__main__":
    main()

Writing ddp_simple.py


In [35]:
!python ddp_simple.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|████████████████████████████████████████| 170M/170M [00:02<00:00, 66.7MB/s]
100%|████████████████████████████████████████| 170M/170M [00:03<00:00, 46.9MB/s]
/kaggle/working/ddp_simple.py:50: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/kaggle/working/ddp_simple.py:57: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
W0618 06:43:17.944000 146 torch/multiprocessing/spawn.py:169] Terminating process 149 via signal SIGTERM
Traceback (most recent call last):
  File "/kaggle/working/ddp_simple.py", line 79, in <module>
    main()
  File "/kaggle/working/ddp_simple.py", line 76, in main
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)
  File "/usr/local/lib/python3.11/dist-packages/torch/multiprocessing/spawn.py", line 340, in spawn
    retu